In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os
import warnings
import seaborn as sns
import re

In [2]:
# load from my local drive
earth_data = pd.read_csv('all_month.csv')
# Description of data property and nature like nun value, dtype total value   
earth_data.info()
# here we count and sum of nun value
earth_data.isnull().sum()
# Here we fill all nun value with there median, it give normalization value during our statistic analysis 
earth_data['mag'] = earth_data['mag'].fillna(earth_data['mag'].median())
earth_data['nst'] = earth_data['nst'].fillna(earth_data['nst'].median())
earth_data['gap'] = earth_data['gap'].fillna(earth_data['gap'].median())
earth_data['dmin'] = earth_data['dmin'].fillna(earth_data['dmin'].median())
earth_data['horizontalError'] = earth_data['horizontalError'].fillna(earth_data['horizontalError'].median())
earth_data['magError'] = earth_data['magError'].fillna(earth_data['magError'].median())
earth_data['magNst'] = earth_data['magNst'].fillna(earth_data['magNst'].median())

In [ ]:
def earthquake_data_on_map(earth_data, screen=None, screen_title='', x='longitude', y='latitude', color_col='net', full_map=True):
    if screen is not None:
        earth_data = earth_data[screen]

    # Set up the figure and the map with Cartopy
    fig, ax = plt.subplots(figsize=(16, 24), subplot_kw={'projection': ccrs.PlateCarree()})
    
    # Set the extent of the map if full_map is True
    if full_map:
        ax.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())

    # Add coastlines and borders
    ax.add_feature(cfeature.COASTLINE, linestyle='-', linewidth=1, edgecolor='grey')
    ax.add_feature(cfeature.BORDERS, linestyle='-', linewidth=1, edgecolor='grey')

    # Create scatter plot using seaborn, with x and y as strings (column names)
    sns.scatterplot(x=x, y=y, data=earth_data, hue=earth_data[color_col], 
                    size=earth_data[color_col], alpha=0.7)

    # Add title and labels
    ax.set_title('Earthquake on Map ' + screen_title)
    ax.text(0.5, -0.05, 'Longitude', va='center', ha='center', transform=ax.transAxes, fontsize=12)
    ax.text(-0.05, 0.5, 'Latitude', va='center', ha='center', rotation='vertical', transform=ax.transAxes, fontsize=12)
    
    # A Gridlines give attractive view and direction label  
    ax.gridlines(draw_labels=True)
    
    plt.legend(title=color_col, loc='best', bbox_to_anchor=(1, 1))

    plt.tight_layout()

    # Display the plot
    plt.show()

# Example call to the function
earthquake_data_on_map(earth_data, screen_title='Location with Station', color_col='net')

In [ ]:
# this function provide accessibility of correlation of earthquake parameters  
def key_factor_correlation(earth_data, screen = None , screen_title = '', diag_kind = 'kde'):
    # Here latitude, longitude and mag correlation are default 
    if screen is  None:
        screen = ['latitude','longitude', 'mag']
    # we can add any other value as well for correlation to each other 
    available_screen = [col for col in screen if col in earth_data.columns]
    # Plot correlation chart with seaborn library
   # fig, ax  = plt.subplots(figsize = (24,16))
    sns.pairplot(earth_data[available_screen], diag_kind='kde',height = 2.5, aspect = 2)
    plt.suptitle(f'Pair Plot of Correlation {screen} ' + screen_title, y = 1.02)
    plt.show()

key_factor_correlation(earth_data)

In [ ]:
# function of pie chart give records percentage wise of each location 
### Here we avoid location of value below 0.4 percent but you can see through Earthquake location count.csv
def earthquake_pie_plot(earth_data):
    # This line split the location form dataset 
    pai_chart_data = earth_data['place'].str.split().str[-1].value_counts()
    # This line avoid a value below <.04 percentage 
    pai_chart_data = pai_chart_data [(pai_chart_data/pai_chart_data.sum() * 100) >= 0.4]
    # This line convert numerical value to percentage 
    explode = [1 if i < 0.05 else 0 for i in pai_chart_data/pai_chart_data.sum()]
    plt.pie(pai_chart_data, explode=explode, labels = pai_chart_data.index, autopct='%1.1f%%', radius= 1.2)
    plt.title('       Pai Chart of Earthquake Location Percentage Records', loc='left')
    plt.show()
    
earthquake_pie_plot(earth_data)